<a href="https://colab.research.google.com/github/shuvookd/Self_Supervised_Learning/blob/main/Self_Supervised_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Self-Supervised Learning

## Introduction
Self-supervised learning (SSL) is a machine learning approach where a model learns meaningful representations from unlabeled data.
Unlike supervised learning, which relies on labeled data, SSL generates its own labels by designing pretext tasks.

## Classification of Self-Supervised Learning
SSL can be categorized into:
1. **Contrastive Learning**: The model learns by comparing similar and dissimilar samples.
   - Examples: SimCLR, MoCo, BYOL.
2. **Generative Approaches**: The model reconstructs missing or corrupted parts of the input.
   - Examples: Autoencoders, Masked Autoencoders (MAE), BERT (in NLP).
3. **Clustering-Based Methods**: The model groups similar data points and learns meaningful representations.
   - Examples: DeepCluster, SwAV.

## Scope of Self-Supervised Learning
Self-supervised learning has applications across various domains, including:
- **Computer Vision**: Pretraining models on large-scale image datasets before fine-tuning them for specific tasks.
- **Natural Language Processing (NLP)**: Models like BERT and GPT use self-supervised learning to understand language without labeled data.
- **Healthcare**: Learning patterns in medical images or genomic sequences without requiring large amounts of labeled data.
- **Speech Processing**: SSL is used in automatic speech recognition (ASR) and speaker verification.
- **Robotics**: Training robots to understand and interact with their environment through self-supervised learning.

#Implementation

## Imports and Setup
First, we import the necessary libraries.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

## Data Preparation
We define transformations to augment the images and load the CIFAR-10 dataset.

In [3]:
transform = transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

100%|██████████| 170M/170M [00:29<00:00, 5.78MB/s]


## Model Definition
We define a simple CNN encoder to extract features from images.

In [4]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.fc = nn.Linear(128 * 8 * 8, 256)

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

## Contrastive Loss Function
The contrastive loss function measures the similarity between image representations.

In [5]:
def contrastive_loss(features1, features2, temperature=0.5):
    similarity_matrix = torch.matmul(features1, features2.T) / temperature
    labels = torch.arange(features1.size(0)).to(features1.device)
    loss = nn.CrossEntropyLoss()(similarity_matrix, labels)
    return loss

## Training the Model
We initialize the model, optimizer, and loss function, then train the model using contrastive learning.

In [6]:
encoder = Encoder()
optimizer = optim.Adam(encoder.parameters(), lr=0.001)
device = "cuda" if torch.cuda.is_available() else "cpu"
encoder.to(device)

epochs = 10
for epoch in range(epochs):
    for images, _ in train_loader:
        images = images.to(device)

        augmented_images1 = images
        augmented_images2 = images

        features1 = encoder(augmented_images1)
        features2 = encoder(augmented_images2)

        loss = contrastive_loss(features1, features2)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

print("Self-supervised learning training completed!")

Epoch [1/10], Loss: 0.0018
Epoch [2/10], Loss: 0.0013
Epoch [3/10], Loss: 0.0087
Epoch [4/10], Loss: 0.0021
Epoch [5/10], Loss: 0.0003
Epoch [6/10], Loss: 0.0000
Epoch [7/10], Loss: 0.1092
Epoch [8/10], Loss: 0.0016
Epoch [9/10], Loss: 0.0004
Epoch [10/10], Loss: 0.0008
Self-supervised learning training completed!
